In [ ]:
import json
import pandas as pd
import requests
import os
import csv
import sqlite3

In [ ]:
#API REQUEST

# API endpoint for disney characters
url = "https://api.disneyapi.dev/character"

# Make the GET request
response = requests.get(url)
data = response.json()

# Check if the request was successful
# Check the status code
if response.status_code == 200:
    print("API request successful!")
else:
    print(f"Failed to retrieve data: {response.status_code}")

API request successful!


In [ ]:
# Creating function to put all the pages together
def getData(url): # (Fetch)
  page = 1
  totalRecords = 0
  allData = []

  while page <= 149:
    response = requests.get(url, params={'page': page})
    data = response.json()

    allData.extend(data['data'])

    # Count all the records on the current page
    totalRecords += len(data['data'])
    page += 1

  return allData, totalRecords

allData, totalRecords = getData(url)



In [ ]:
# Counting the number of columns and records (Pre)

numColumns = len(allData[0].keys())
print(f"Number of columns before modification: {numColumns}")

print(f"Number of records before modification: {totalRecords}")

Number of columns before modification: 15
Number of records before modification: 7438


In [ ]:
# Titles of the columns
print(allData[0])

{'_id': 112, 'films': ['Hercules (film)'], 'shortFilms': [], 'tvShows': ['Hercules (TV series)'], 'videoGames': ['Kingdom Hearts III'], 'parkAttractions': [], 'allies': [], 'enemies': [], 'sourceUrl': 'https://disney.fandom.com/wiki/Achilles_(Hercules)', 'name': 'Achilles', 'imageUrl': 'https://static.wikia.nocookie.net/disney/images/d/d3/Vlcsnap-2015-05-06-23h04m15s601.png', 'createdAt': '2021-04-12T01:31:30.547Z', 'updatedAt': '2021-12-20T20:39:18.033Z', 'url': 'https://api.disneyapi.dev/characters/112', '__v': 0}


In [ ]:
# Removing columns (Modify)
columsRemoved = ['imageUrl', 'videoGames','shortFilms', 'createdAt']
for record in allData:
  for column in columsRemoved:
      if column in record:
          del record[column]

In [ ]:
# Showing the data to confirm that the columns have been removed (Post)
print(allData[0])

{'_id': 112, 'films': ['Hercules (film)'], 'tvShows': ['Hercules (TV series)'], 'parkAttractions': [], 'allies': [], 'enemies': [], 'sourceUrl': 'https://disney.fandom.com/wiki/Achilles_(Hercules)', 'name': 'Achilles', 'updatedAt': '2021-12-20T20:39:18.033Z', 'url': 'https://api.disneyapi.dev/characters/112', '__v': 0}


In [ ]:
# Asking user input for converting the JSON file
print("Would you like to convert the JSON file to a CSV file then SQL file or a SQL file directly?")
question = input("Please enter either directSQL or CSV: ")

if question == 'CSV':
  df = pd.DataFrame(allData)
  df.to_csv('output.csv', index=False)

  df = pd.read_csv('output.csv')


  num_records_after = len(df)
  num_columns_after = len(df.columns)
  print(f"Number of records after conversion: {num_records_after}")
  print(f"Number of columns after conversion: {num_columns_after}")

  connect = sqlite3.connect('output.db')
  df.to_sql('disney_characters', connect, if_exists='replace', index=False) # (Store)



  connect.close()
  print("CSV file created and SQL file created")

elif question == 'directSQL':
  df = pd.DataFrame(data['data'])
  df = df.map(lambda x: str(x) if isinstance(x, (list, dict)) else x)

  num_records_after = len(df)
  num_columns_after = len(df.columns)
  print(f"Number of records after conversion: {num_records_after}")
  print(f"Number of columns after conversion: {num_columns_after}")

  connect = sqlite3.connect('output.db')

  df.to_sql('output', connect, if_exists='replace', index=False)

  connect.close()
  print("SQL file created")

else:
  print("Please enter either directSQL or CSV exactly as they appear")



Would you like to convert the JSON file to a CSV file then SQL file or a SQL file directly?
Please enter either directSQL or CSV: CSV
Number of records after conversion: 7438
Number of columns after conversion: 12
CSV file created and SQL file created


In [ ]:
#CSV FILE - Top 100 Healthiest Foods in the World; 9 columns x 99 rows

file = 'Top 100 Healthiest Food in the World.csv'
# Load Data from Top 100 Healthiest Foods
healthyFoodsDF = pd.read_csv('Top 100 Healthiest Food in the World.csv', sep=',')

# Display the first few foods
print("Healthiest Foods in the World:")
display(healthyFoodsDF.head())

# initializing the titles and rows list
columns = []
rows = []

# reading csv file
with open(file, 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)

    # extracting columns
    columns = next(csvreader)

    # extracting each data row one by one
    for row in csvreader:
        rows.append(row)

    # get total number of rows
    print("Total number of rows: %d" % (csvreader.line_num))

# printing the field names
print("Number of Columns:", len(columns))



Healthiest Foods in the World:


,Food,Nutrition Value (per 100g),Quantity,Originated From,Calories,Protein (g),Fiber (g),Vitamin C (mg),Antioxidant Score
0,Kale,"High in Vitamins A, C, K","1 cup, chopped",Ancient Greece,49,4.3,3.6,93.4,1770
1,Blueberries,High in Antioxidants,1 cup,North America,84,1.1,3.6,14.4,9621
2,Salmon,Rich in Omega-3 Fatty Acids,3 oz,North Pacific Ocean,208,22.1,0.0,0.0,689
3,Garlic,Contains Allicin,1 clove,Central Asia,4,0.2,0.1,0.9,5708
4,Spinach,High in Iron and Folate,"1 cup, raw",Ancient Persia,23,2.9,2.2,28.1,1515


Total number of rows: 99
Number of Columns: 9


In [ ]:
# Dropping the Antioxidant Score
healthyFoodsDF.drop('Antioxidant Score', axis=1, inplace=True)

# Display the first few foods
print("Healthiest Foods in the World:")
display(healthyFoodsDF.head())

Healthiest Foods in the World:


,Food,Nutrition Value (per 100g),Quantity,Originated From,Calories,Protein (g),Fiber (g),Vitamin C (mg)
0,Kale,"High in Vitamins A, C, K","1 cup, chopped",Ancient Greece,49,4.3,3.6,93.4
1,Blueberries,High in Antioxidants,1 cup,North America,84,1.1,3.6,14.4
2,Salmon,Rich in Omega-3 Fatty Acids,3 oz,North Pacific Ocean,208,22.1,0.0,0.0
3,Garlic,Contains Allicin,1 clove,Central Asia,4,0.2,0.1,0.9
4,Spinach,High in Iron and Folate,"1 cup, raw",Ancient Persia,23,2.9,2.2,28.1


In [ ]:
# Changing dataframe to include only top 50 foods
healthyFoodsDF = healthyFoodsDF[:-47]

# Display the first few foods
print("Top 50 Healthiest Foods in the World:")
display(healthyFoodsDF)

Top 50 Healthiest Foods in the World:


,Food,Nutrition Value (per 100g),Quantity,Originated From,Calories,Protein (g),Fiber (g),Vitamin C (mg)
0,Kale,"High in Vitamins A, C, K","1 cup, chopped",Ancient Greece,49,4.3,3.6,93.4
1,Blueberries,High in Antioxidants,1 cup,North America,84,1.1,3.6,14.4
2,Salmon,Rich in Omega-3 Fatty Acids,3 oz,North Pacific Ocean,208,22.1,0.0,0.0
3,Garlic,Contains Allicin,1 clove,Central Asia,4,0.2,0.1,0.9
4,Spinach,High in Iron and Folate,"1 cup, raw",Ancient Persia,23,2.9,2.2,28.1
5,Almonds,Good Source of Vitamin E,1 oz (23 nuts),Middle East,164,6.0,3.5,0.0
6,Broccoli,High in Vitamin C and K,"1 cup, chopped",Italy,31,2.5,2.4,81.2
7,Avocado,Rich in Healthy Fats,1 medium,Central America,320,4.0,13.5,10.0
8,Quinoa,Complete Protein Source,"1 cup, cooked",South America,222,8.1,5.2,0.0
9,Lentils,High in Protein and Fiber,"1 cup, cooked",Middle East,230,17.9,15.6,2.1


In [ ]:
# Addtional column: My personal rating of the taste of each food on a scale from 1-10
rating = [5, 7, 8, 10, 5, 6, 7, 10, 4, 5, 4, 6, 4, 9, 5, 8, 7, 6, 5, 5, 1, 2, 5, 7, 7, 5, 4, 4, 6, 6, 6, 8, 10, 9, 7, 3, 6, 4, 3, 9, 10, 7, 7, 7, 6, 6, 5, 5, 4, 8, 7]
healthyFoodsDF.insert(8, "Teagans Taste Rating", rating)

# Display the first few foods
print("Top 50 Healthiest Foods in the World:")
display(healthyFoodsDF.head())

# disply number of rows in altered databse
print("Number of Rows After Alteration:", len(healthyFoodsDF))

# display number of columns in altered databse
print("Number of Columns After Alteration:", len(healthyFoodsDF.columns))

Top 50 Healthiest Foods in the World:


,Food,Nutrition Value (per 100g),Quantity,Originated From,Calories,Protein (g),Fiber (g),Vitamin C (mg),Teagans Taste Rating
0,Kale,"High in Vitamins A, C, K","1 cup, chopped",Ancient Greece,49,4.3,3.6,93.4,5
1,Blueberries,High in Antioxidants,1 cup,North America,84,1.1,3.6,14.4,7
2,Salmon,Rich in Omega-3 Fatty Acids,3 oz,North Pacific Ocean,208,22.1,0.0,0.0,8
3,Garlic,Contains Allicin,1 clove,Central Asia,4,0.2,0.1,0.9,10
4,Spinach,High in Iron and Folate,"1 cup, raw",Ancient Persia,23,2.9,2.2,28.1,5


Number of Rows After Alteration: 51
Number of Columns After Alteration: 9


In [ ]:
#CSV Conversion (Convert)
askForm = print('Would you like to convert the CSV to a SQL file directly or to JSON file then SQL file?')
form = input('Please enter either direct or JSON: ')


#Convert CSV to JSON
if form == 'JSON':
  healthyFoodsDF.to_json('Top 100 Healthiest Foods in the World.json')
  healthyFoodsDF.to_sql('Top 100 Healthiest Foods in the World.db', con=sqlite3.connect('Top 100 Healthiest Foods in the World.db'))
  print('CSV converted to SQL')
#Convert CSV to SQL
elif form == 'direct':
  healthyFoodsDF.to_sql('Top 100 Healthiest Foods in the World.db', con=sqlite3.connect('Top 100 Healthiest Foods in the World.db'))
  print('CSV converted to SQL')
#Error mitigation if user enters something else
else:
  print('Please enter either direct or JSON exactly as they appear')


Would you like to convert the CSV to a SQL file directly or to JSON file then SQL file?
Please enter either direct or JSON: direct
CSV converted to SQL
